Pasos:

Extraer con cpptraj las trayectorias con únicamente los CA y proteina

% Ya en eel nootebook de pcas

PAra cada ensayo:

- Carga la conformación inicial
- Carga las trayectorias (5 de 20 ns)
- para PCA
    - Alinear con respecto a la inicial (la estructura cristalográfica inicial)
         - la idea es que la incial es un aestructura cristalográfica y está proyección original
    - Alinear todas las conformaciones de la producción, de tal manera que el clustering y el PCA permita extraer las conformaciones centroides de únicamente este espacio
    - Usar los residuos definidos por Pisani
    - Usar los residuos de la estructura secundaria
    - Usar los residuos del pocket
- Proyección con MDS
    - Las estruturas se alinean con respecto al frame 1 de las cristalográficas
    - Revisar el notebook proyeccion, dejamos pendiente por ahora esta parte
 

In [14]:
import subprocess
import os

def strip_cpptraj(system_name, # (nombre del sistema, corresponde a la carpeta principal del mismo)
                 md_type, # tipo de simulación realizada (sa, mxs, etc)
                 n_rep, # Número de réplica, asumiendo que el directorio se llama 'rep_n'
                 file_system_name, # Sufijo de los archivos prmtop, rst7 y nc
                 DIR_DM = '../../ARCHIVOS/DINAMICA', # Directorio general de las dinámicas
                 DIR_prod_name = '4_prod'
                 ):
    prevdir = os.getcwd()
    # Determina el directorio del archivo de producción
    # Se asume que la carpeta se llama 3_prod o 4_prod
    DIR_REP_PRODUCTION = F'{DIR_DM}/{system_name}/{md_type}/rep_{n_rep}/{DIR_prod_name}'
    # Crea el archivo para su ejecuación con cpptraj
    with open(F'{DIR_REP_PRODUCTION}/run_strip.cpptraj', 'w') as f:
        f.write(F'''# Strip CA atoms from traj
parm ../{file_system_name}.prmtop
trajin {file_system_name}_prod.nc
reference ../{file_system_name}.rst7
strip :WAT:Cl-:Na+:ETA outprefix PROT
trajout ./PROT.{file_system_name}_prod.nc
go
strip !(@CA) outprefix CA
trajout ./CA.{file_system_name}_prod.nc
go
    ''')
    os.chdir(DIR_REP_PRODUCTION)
    try:
        os.system('cpptraj -i run_strip.cpptraj')
    except ValueError as e:
        print('Hubo un error en la ejecución', e)
    finally:
        os.chdir(prevdir)

In [15]:
for i in range(1, 6):
    strip_cpptraj('4fku', 'mix_solv', i, 'mxs_cdk2')

In [10]:
%ls

Untitled.ipynb


In [24]:
DIR_REP_PRODUCTION = F'{DIR_REP}/4_prod'

f_prmtop = F'../{file_system_name}.prmtop'
f_rst7 = F'../{file_system_name}.rst7'
f_nc = F'{file_system_name}_prod.nc'

In [38]:
# Se crea el archivo de entrada para cpptraj
with open(F'{DIR_REP_PRODUCTION}/run_strip.cpptraj', 'w') as f:
    f.write(F'''# Strip CA atoms from traj
parm {f_prmtop}
trajin {f_nc}
reference {f_rst7}
strip :WAT:Cl-:Na+:ETA outprefix PROT
trajout ./PROT.{f_nc}
strip !(@CA) outprefix CA
trajout ./CA.{f_nc}
go
    ''')

# Pasos
Crra un archivo script

In [4]:
%ls

Untitled.ipynb


In [ ]:
import pandas as pd
import numpy as np
import pytraj as pyt